In [131]:
import os
import pandas as pd
from glob import glob
import numpy
import fnmatch
import pickle
from deepdiff import DeepDiff


In [2]:
raw_beh_path = os.path.abspath(r'C:\Users\USER\Guro_Psy_KJH Dropbox\1.Projects\1_anxiety_VR\3_Data\1_Behavior\0_raw')

In [173]:
# Input: local location for raw behavior data from VRABES
# Output: dictionary, key = sub-xxxx_ses-xx, value = list of path 
# reference: ref1 : 폴더명 생성 규칙 
def gen_dic_paths (raw_beh_path): 
    dic_path = {}  
    subs = os.listdir(raw_beh_path) 
    for sub in subs:
        path2 = os.path.join(raw_beh_path,sub)
        sess = os.listdir(path2) 
        for ses in sess:
            path3 = os.path.join(path2,ses)
            paths = os.listdir(path3) 
            dic_path['_'.join([sub,ses])] = paths           
                
    return dic_path

In [174]:
# Input: return from gen_dic_paths
# output: dictionary, key = sub-xxxx_ses-xx, value = list of filename
# reference: ref2 : Information and location by file

def gen_dic_flist ():
    a = gen_dic_paths(raw_beh_path)
    dic_file = {}    
    for key in a.keys():        
        f_list=[]
        for i in a[key]:
            path_list = [raw_beh_path, key.split('_')[0], key.split('_')[1],i]      
            p = os.path.join(*path_list)
            f_list.extend(glob(p+'/*.csv'))
            f_list.extend(glob(p+'/*.txt'))                      
            
        dic_file[key] = f_list
    return dic_file

In [175]:
# 개발중 - 추후에 추가되는 실험 내용에 대하여 반영이 가능하게 고민 중
def old_dict_ex():
    with open('vrabes_beh_dict.pkl','rb') as f:
        old_dict = pickle.load(f)
    ex_old_dict = {}
    for key in old_dict.keys():
        col_list = old_dict[key].keys()
        flist = []
        for a in col_list:
            col_names = a.split('_')[:-1]
            fname = '_'.join(col_names)
            if fname in flist:
                pass
            else: flist.append(fname)
        ex_old_dict[key] = flist
    return ex_old_dict

In [176]:
# 개발중 - 추후에 추가되는 실험 내용에 대하여 반영이 가능하게 고민 중 - main 함수를 차이가 있는데서만 작동하도록
# Input: pkl - generated dictionary from gen_df
# Output: dictionary, Deepdiff -> ref4
# Compares the dictionary generated by gen_dic_flist() of raw_beh_path at execution time 
# and the flist dictionary extracted from the existing storage dictionary from "old_dict_ex" 
# and returns the difference

def diff_dic ():
    new_dict = gen_dic_flist()
    ex_new_dict = {}
    for key in new_dict.keys():
        new_flist =[]
        for fname in new_dict[key]:
            new_flist.append(fname.split('\\')[-1][:-4])
        ex_new_dict[key] = new_flist

    old_dict = old_dict_ex()
    diff = DeepDiff(old_dict,ex_new_dict,verbose_level=2)
    return diff
    
    
        

In [177]:
# Input: string from HRV_Result
# output: string, name & val 
# reference: ref3 : Information and location by file
def hrv_r_slicer (x):
    eq_ind = x.find('=') 
    name = x[:eq_ind]
    after_eq = x[eq_ind+1:] 
    if '(' in after_eq:
        br_ind = after_eq.find('(')
        val = x[eq_ind+2:eq_ind+br_ind+1] 
    else:
        val = x[eq_ind+2:]
    return name, val

In [110]:
# Input: generate pickle
# output: dictionary, key = sub-xxxx_sex-xx, value = dataframe for each experiment

def gen_final_dict ():
    f_path = gen_dic_flist()
    result = {}
    for key in f_path.keys():
        print(key)        
        dic_df = {}       
        for i in f_path[key]:            
            i_sp = i.split('\\')[-1]
            if i_sp[-3:] == 'csv':
                r_df = pd.read_csv(i,header=1)
                col1 = i_sp[:-4]
                if i_sp[-7:-4] =='HRV':
                    col2 = r_df.columns[0]
                    val = r_df.columns[1]
                    dic_df['_'.join([col1,col2])] = val
                else:
                    for d in r_df.columns:
                        val = r_df[d][0]
                        dic_df['_'.join([col1,d])] = val  
            elif i_sp[-3:] == 'txt':
                r_df = pd.read_csv(i,header=None)
                col1 = i_sp[:-4]
                for e in r_df.loc[:,0]:
                    col2, val = hrv_r_slicer(e)
                    dic_df['_'.join([col1,col2])] = val
        result[key] = dic_df
    return dic_df
 

In [178]:
def main():
    dic_df = gen_final_dict()
    with open('vrabes_beh_dict.pkl','wb') as f:
        pickle.dump(dic_df,f)   